https://www.fukushihoken.metro.tokyo.lg.jp/hodo/saishin/hassei.html


In [1]:
import os
from WebScrapingTool import Base_UserFunction as uf
import json
import urllib

In [2]:
# https://www.shibutan-bloomers.com/python_library_pdfminer-six/2124/#21PDFJupyterNotebook

from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
from io import StringIO
import jaconv
from datetime import date
from japanera import Japanera, EraDate
import unicodedata
import copy

janera = Japanera()

def parse(filePath, type):
    getText = ''
    retData = '{'
    try:
        fp = open(filePath, 'rb')
        outfp = StringIO()

        rmgr = PDFResourceManager()
        lprms = LAParams()
        device = TextConverter(rmgr, outfp, laparams=lprms)
        iprtr = PDFPageInterpreter(rmgr, device)

        for page in PDFPage.get_pages(fp):
            iprtr.process_page(page)
        getText = outfp.getvalue()
    finally:
        device.close()
        outfp.close()
        fp.close()

    baseList = getText.split()

    idx = -1
    modeSokuhou = (
        '患者の発生状況', 
        '行政検査数', 
        '都の独自検査件数', 
        '10歳未満', 
        '都内発生数', 
        '区市町村別患者数' 
    )

    modeAdd = (
        '患者の発生状況', 
        '都内発生数', 
        '重症者の属性',
        '入院・療養等調整中の陽性患者について' 
    )


    endWord = '都内コロナウイルス陽性患者'

    mode = list()
    if baseList[0] in '別紙':
        retData += '"isAddData" : "False", ' 
        mode = copy.deepcopy(modeSokuhou)
    else:
        retData += '"isAddData" : "True", ' 
        mode = copy.deepcopy(modeAdd)
        
    # date
    _date = jaconv.z2h(baseList[1], kana=False, ascii=False, digit=True)
    _date = _date.replace('◆', '')
    _date = _date[:_date.find('日') + 1]
    _date = janera.strptime(_date, "%-E%-kO年%-km月%-kd日")
    _date = _date[0].strftime('%Y/%m/%d')
    print(_date)
    retData += '"date" : "' + _date
    
    for l in baseList[2:]:
        if endWord in l:
            break

        if idx < len(mode) - 1 and mode[idx + 1] in l:
            idx += 1
            retData += '", "' + mode[idx] + '" : "'
        
        tmp = l.replace('(', '').replace(')', '').replace(',', '').replace('%', '')
        if tmp.encode('utf-8').isdigit():
            retData += tmp + ","
        elif tmp.isascii() and '.' in tmp:
            retData += tmp + ","

    retData += '"}'
    print(retData)
    return retData


In [3]:
def main():
    print("\n[Start]"  + uf.getNowTime() + '\n')

    #設定ファイルから必要な情報を取得する
    #タグ
    tag_debug = '[a]'
    tag_saveFolder = '[b]'
    tag_loadFileName = '[c]'
    tag_saveFileName = '[j]'
    
    isDebug = False
    _saveFolder = ''
    _loadFileName = ''
    _saveFileName = ''
    
    try:
        with open('_Setting.txt', mode='r') as f:
            lines = f.readlines()
            for l in lines:
                if l.startswith(tag_debug, 0, 3):
                    if (l.replace(tag_debug, '').rstrip()).lower() == 'true':
                        isDebug = True
                    else:
                        isDebug = False                    

                if l.startswith(tag_saveFolder, 0, 3):
                    _saveFolder = l.replace(tag_saveFolder, '').rstrip()
             
                if l.startswith(tag_loadFileName, 0, 3):
                    _loadFileName = l.replace(tag_loadFileName, '').rstrip()
             
                if l.startswith(tag_saveFileName, 0, 3):
                    _saveFileName = l.replace(tag_saveFileName, '').rstrip()
             
    except:
        print('[!!!ERROR!!!] Read Setting.text')
        return        
    
    if len(_saveFolder) <= 0:
        print('[!!!ERROR!!!] Image data storage folder is None!')
        return  
    
    baseFile =_saveFolder + "/" + _loadFileName
    print(baseFile)

    saveFile =_saveFolder + "/" + _saveFileName
    print(saveFile)
    
    # ファイルを開く
    updateList = list()
    parseList = list()
    with open(baseFile, mode='r') as f:
        cnt = 0
        for line in f:
            l = line
            j = json.loads(line)
            type = j['type']
            fileName = j['name']
            isParse = j['isParse']
            if isParse == "False":
                parseList.append(parse(_saveFolder + "/" + fileName, type) + "\n")
                cnt += 1
                print("...Parse PDF : " + fileName + '  ' + str(cnt))
                l = l.replace('"isParse" : "False"', '"isParse" : "True"')
            updateList.append(l)

        print('\n...Get Size :' + str(cnt) + '\n')

    # ファイル更新
    with open(baseFile, mode='w') as f:
        for line in updateList:
                uf.fileWrite(f, line)

    # パースしたデータを追加
    with open(saveFile, mode='a') as f:
        for line in parseList:
                uf.fileWrite(f, line)

    print("\n[ End ]"  + uf.getNowTime() + '\n')
    
    
if __name__ == '__main__':
    main()


[Start]20210725104411

#data/dataList.json
#data/outputList.csv
2021/07/15
{"isAddData" : "False", "date" : "2021/07/15", "患者の発生状況" : "1308,766,542,0,45,13,882.1,663.6,132.9,", "行政検査数" : "10388.7,13290,9553,8323,", "都の独自検査件数" : "667,541,23,29001,5857,3163,5475,43496,6214,", "10歳未満" : "50,103,450,256,211,153,50,20,10,5,0,0,", "都内発生数" : "185427,57,", "区市町村別患者数" : "970,910,2975,2854,5965,5763,9891,9685,2794,2611,3385,3213,3760,3549,6337,5931,5424,5164,5046,4858,9017,8545,13422,12740,5236,5058,6069,5760,7452,6970,4890,4689,4223,4006,2974,2852,6934,6607,8091,7747,8433,8039,5988,5798,7871,7528,4339,4234,1499,1432,1449,1376,1739,1648,855,827,2071,2004,866,828,2148,2051,2965,2847,1062,1016,1261,1199,1259,1233,924,887,921,871,499,471,518,499,672,643,499,478,441,424,745,700,406,383,1012,978,594,576,361,353,592,575,1795,1715,191,186,112,108,10,10,25,25,52,24,0,0,2,2,1,1,5,4,2,2,9,9,0,0,4,4,14987,14863,80,78,"}
...Parse PDF : 20210716_0716-20-01.pdf  1
2021/07/18
{"isAddData" : "False", "date" : 